# Resources
https://docs.google.com/document/d/11MFXRQfpPkBS37qSlBGKb5EGD0DH7hEtKu-YlV5sK0I/edit

https://towardsdatascience.com/how-to-train-test-split-kfold-vs-stratifiedkfold-281767b93869?gi=194ee6e79b9d

In [1]:
import re

# Get pandas and postgres to work together
import psycopg2 as pg
import pandas as pd

# We are also going to do some basic viz
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline

In [2]:
plt.rcParams['figure.figsize'] = (9, 6)
sns.set(context='notebook', style='whitegrid', font_scale=1.2)

In [3]:
# Postgres info to connect
connection_args = {
    'host': 'localhost',  # We are connecting to our _local_ version of psql
    'dbname': 'myersbriggs',    # DB that we are connecting to
    'port': 5432          # port we opened on AWS
}

connection = pg.connect(**connection_args)  # What is that "**" there??
cur = connection.cursor()

## Read data from .csv
```python
df = pd.read_csv('mbti_1.csv')
```

## Create a table in PostgreSQL database

```python
sqlCreateTable = "create table raw_data (id bigint, type varchar(4), posts text);"

cur.execute(sqlCreateTable)
connection.commit()

# Didn't work.  Created manually.

def make_SQL_string():
    columns = ["post_" + str(i) + " text, " for i in range(58)]

    return "CREATE TABLE posts;"

cur.execute(make_SQL_string())
connection.commit()
```

## Inserting the data into the database.
```python
def single_insert(conn, insert_req, values):
    """ Execute a single INSERT request """
    cursor = conn.cursor()
    try:
        cursor.execute(insert_req, values)
        conn.commit()
    except (Exception, pg.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

    # Inserting each row
for i in df.index:

    query = "INSERT into raw_data(id, type, posts) values('%s',%s,%s);"
    values = (i, df['type'][i], df['posts'][i])
    single_insert(connection, query, values)
```

In [4]:
postgreSQL_select_Query = "SELECT * FROM raw_data"

cur.execute(postgreSQL_select_Query)
print("Selecting rows from mobile table using cursor.fetchall")
myers_briggs = cur.fetchall()

Selecting rows from mobile table using cursor.fetchall


In [8]:
mb_df = pd.DataFrame(myers_briggs)

post_list = [re.split('\|\|\|+', post) for post in mb_df[1]]
post_df = pd.DataFrame(post_list)
post_df.insert(loc=0, column='type', value=mb_df[0])

In [63]:
cur.close()
# Close the connection
connection.close()

In [59]:
def single_insert(conn, insert_req, values):
    """ Execute a single INSERT request """
    cursor = conn.cursor()
    try:
        cursor.execute(insert_req, values)
        conn.commit()
    except (Exception, pg.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

    # Inserting each row
for i in mb_df.index:

    query = "INSERT into posts(post_0, post_1, post_2, post_3, post_4, post_5, post_6, post_7, post_8, post_9, post_10, post_11, post_12, post_13, post_14, post_15, post_16, post_17, post_18, post_19, post_20, post_21, post_22, post_23, post_24, post_25, post_26, post_27, post_28, post_29, post_30, post_31, post_32, post_33, post_34, post_35, post_36, post_37, post_38, post_39, post_40, post_41, post_42, post_43, post_44, post_45, post_46, post_47, post_48, post_49, post_50, post_51, post_52, post_53, post_54, post_55, post_56, post_57, posts_58) values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
    values = (mb_df['0'], mb_df['1'], mb_df['2'], mb_df['3'], mb_df['4'], mb_df['5'], mb_df['6'], mb_df['7'], mb_df['8'], mb_df['9'], mb_df['10'], mb_df['11'], mb_df['12'], mb_df['13'], mb_df['14'], mb_df['15'], mb_df['16'], mb_df['17'], mb_df['18'], mb_df['19'], mb_df['20'], mb_df['21'], mb_df['22'], mb_df['23'], mb_df['24'], mb_df['25'], mb_df['26'], mb_df['27'], mb_df['28'], mb_df['29'], mb_df['30'], mb_df['31'], mb_df['32'], mb_df['33'], mb_df['34'], mb_df['35'], mb_df['36'], mb_df['37'], mb_df['38'], mb_df['39'], mb_df['40'], mb_df['41'], mb_df['42'], mb_df['43'], mb_df['44'], mb_df['45'], mb_df['46'], mb_df['47'], mb_df['48'], mb_df['49'], mb_df['50'], mb_df['51'], mb_df['52'], mb_df['53'], mb_df['54'], mb_df['55'], mb_df['56'], mb_df['57'], mb_df['58'])
    single_insert(connection, query, values)

KeyError: '0'

In [58]:
def make_SQL_string():
    columns = ["mb_df['" + str(i) + "'], " for i in range(58)]

    return "CREATE TABLE posts (" + "".join(columns) + "posts_58)"

make_SQL_string()

"CREATE TABLE posts (mb_df['0'], mb_df['1'], mb_df['2'], mb_df['3'], mb_df['4'], mb_df['5'], mb_df['6'], mb_df['7'], mb_df['8'], mb_df['9'], mb_df['10'], mb_df['11'], mb_df['12'], mb_df['13'], mb_df['14'], mb_df['15'], mb_df['16'], mb_df['17'], mb_df['18'], mb_df['19'], mb_df['20'], mb_df['21'], mb_df['22'], mb_df['23'], mb_df['24'], mb_df['25'], mb_df['26'], mb_df['27'], mb_df['28'], mb_df['29'], mb_df['30'], mb_df['31'], mb_df['32'], mb_df['33'], mb_df['34'], mb_df['35'], mb_df['36'], mb_df['37'], mb_df['38'], mb_df['39'], mb_df['40'], mb_df['41'], mb_df['42'], mb_df['43'], mb_df['44'], mb_df['45'], mb_df['46'], mb_df['47'], mb_df['48'], mb_df['49'], mb_df['50'], mb_df['51'], mb_df['52'], mb_df['53'], mb_df['54'], mb_df['55'], mb_df['56'], mb_df['57'], posts_58)"

In [51]:
make_SQL_string()

'CREATE TABLE posts (post_0, post_1, post_2, post_3, post_4, post_5, post_6, post_7, post_8, post_9, post_10, post_11, post_12, post_13, post_14, post_15, post_16, post_17, post_18, post_19, post_20, post_21, post_22, post_23, post_24, post_25, post_26, post_27, post_28, post_29, post_30, post_31, post_32, post_33, post_34, post_35, post_36, post_37, post_38, post_39, post_40, post_41, post_42, post_43, post_44, post_45, post_46, post_47, post_48, post_49, post_50, post_51, post_52, post_53, post_54, post_55, post_56, post_57, posts_58)'